In [1]:
import numpy as np
%load_ext wurlitzer
%matplotlib inline
import matplotlib.pyplot as plt
import json


import energyflow as ef

In [3]:
!pip install pot

In [2]:
def load_graphs(file_path):
    with open(file_path, 'r') as file:
        graphs = json.load(file)
    return graphs

In [3]:
def get_pair():
    hadron_graph = 'hadron_graphs.json'
    parton_graph = 'parton_graphs.json' ## change to gluon collision or parton if needed

    hadron_graph_data = load_graphs(hadron_graph)
    parton_graph_data = load_graphs(parton_graph)

    return hadron_graph_data, parton_graph_data

In [6]:
## 1- Import data as a Json
## 2 - Convert the data into Gs and Qs
## 3 - Plot data

## Expanding on 2 - 
## The Python code needs the data as Gs and Qs Matricies
## It takes the first three columns which are trans momentum(pT), rapidity and azmuthius angle
## Gs and Qs are lists of Events so each index is a different collision
## 
## Start by creating Gs and Qs and then while parsing through them only add the three features that we need
## From the C++ code the features are 

## Stored This way in the C++ to use as reference when indexing in the Python code
# features.push_back(pythia.event[i].id());
# features.push_back(pythia.event[i].px());
# features.push_back(pythia.event[i].py());
# features.push_back(pythia.event[i].pz());
# features.push_back(pythia.event[i].e());
# features.push_back(pythia.event[i].eta());
# features.push_back(pythia.event[i].phi());
# peudorapidity


In [7]:
Gs, Qs = [], []
hadron_graph_data, parton_graph_data = get_pair()

# print(hadron_graph_data[0]['node_features'][0])

# event feautures particle

for hadron_event, parton_event in zip(hadron_graph_data, parton_graph_data):

    new_hadron_event = []
    new_parton_event = []

    new_hadron_particle = []
    new_parton_particle = []

    
    # print(hadron_event['node_features'])  # To see the structure of the array




    for hadron_particle, parton_particle in zip(hadron_event['node_features'], parton_event['node_features']):
        
        hadron_pT = (hadron_particle[1]**2 + hadron_particle[2]**2)**0.5
        parton_pT = (parton_particle[1]**2 + parton_particle[2]**2)**0.5

        new_hadron_particle = np.array([hadron_pT, hadron_particle[7], hadron_particle[6]])
        new_parton_particle = np.array([parton_pT, parton_particle[7], parton_particle[6]])

        new_hadron_event.append(new_hadron_particle)
        new_parton_event.append(new_parton_particle)

    Gs.append(np.array(new_hadron_event))
    Qs.append(np.array(new_parton_event))




In [8]:
print(Gs)

[array([[7.22051685e+01, 2.18188941e-01, 4.36238974e-01],
       [2.78891489e+02, 2.19461262e-01, 4.37777638e-01],
       [2.71391783e+02, 2.20757872e-01, 4.37001765e-01],
       [1.03967939e+01, 2.02405900e-01, 4.21846211e-01],
       [2.06122983e+01, 2.08381742e-01, 4.29377496e-01],
       [1.50861525e+00, 2.82259136e-01, 3.40484917e-01],
       [1.42250587e+00, 2.54273087e-01, 5.64885736e-01],
       [2.69347050e+00, 1.97425574e-01, 5.77084422e-01],
       [2.45279273e+00, 1.18775927e-01, 6.09149158e-01],
       [1.98731010e+00, 2.77093351e-01, 6.44352555e-01],
       [4.22025725e+00, 1.68424696e-02, 7.18936861e-01],
       [1.88499655e+01, 2.33500034e-01, 4.46492791e-01],
       [6.61068475e+00, 2.62537569e-01, 4.98370409e-01],
       [4.28536162e+01, 2.03974381e-01, 6.12675786e-01],
       [5.16082350e+01, 2.08749801e-01, 6.29484534e-01],
       [2.18069478e+01, 1.95249617e-01, 6.12571597e-01],
       [5.35454557e+01, 1.89411849e-01, 6.18126988e-01],
       [1.35245421e+01, 1.6750

In [9]:
# COPY AND PASTED FROM DEMO CODE
ev0, ev1 = Gs[0], Gs[15]

# calculate the EMD and the optimal transport flow
R = 0.4
emdval, G = ef.emd.emd(ev0, ev1, R=R, return_flow=True)

# plot the two events
colors = ['red', 'blue']
labels = ['Gluon Jet 1', 'Gluon Jet 2']
for i,ev in enumerate([ev0, ev1]):
    pts, ys, phis = ev[:,0], ev[:,1], ev[:,2]
    plt.scatter(ys, phis, marker='o', s=2*pts, color=colors[i], lw=0, zorder=10, label=labels[i])
    
# plot the flow
mx = G.max()
xs, xt = ev0[:,1:3], ev1[:,1:3]
for i in range(xs.shape[0]):
    for j in range(xt.shape[0]):
        if G[i, j] > 0:
            plt.plot([xs[i, 0], xt[j, 0]], [xs[i, 1], xt[j, 1]],
                     alpha=G[i, j]/mx, lw=1.25, color='black')

# plot settings
plt.xlim(-R, R); plt.ylim(-R, R)
plt.xlabel('Rapidity'); plt.ylabel('Azimuthal Angle')
plt.xticks(np.linspace(-R, R, 5)); plt.yticks(np.linspace(-R, R, 5))

plt.text(0.6, 0.03, 'EMD: {:.1f} GeV'.format(emdval), fontsize=10, transform=plt.gca().transAxes)
plt.legend(loc=(0.1, 1.0), frameon=False, ncol=2, handletextpad=0)

plt.show()

ImportError: dlopen(/Users/michaeltaleb/myenv/lib/python3.12/site-packages/wasserstein/_wasserstein_omp.cpython-312-darwin.so, 0x0002): symbol not found in flat namespace '___kmpc_barrier'